In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
import time
import threading


from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
from time import sleep
import pandas as pd
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
cursor = conn.cursor()
port = 7496

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.orderId = 0
        self.list_ = []
        etf = pd.read_csv("a.csv")
        self.etfList = list(etf.code)[:]
        self.index = 0
        self.curCode = self.etfList[self.index]
    def nextValidId(self, orderId):
        self.orderId = orderId
    def nextId(self):
        self.orderId += 1
        return self.orderId
    
    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")

    def historicalData(self, reqId, bar):
        bar = bar.to_dict()
        bar['symbol'] = self.curCode
        self.list_.append(bar)
    def historicalDataEnd(self, reqId, start, end):
        self.cancelHistoricalData(reqId)
        self.index += 1
        self.curCode = self.etfList[self.index]
    def loop(self):
        for i in self.etfList:
            mycontract = Contract()
            mycontract.symbol = i
            mycontract.secType = "STK"
            mycontract.exchange = "SMART"
            mycontract.currency = "USD"
            nextId = app.nextId()
            app.reqHistoricalData(nextId, mycontract, "20240120 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
app = TestApp()
app.connect("127.0.0.1", port, 0)
threading.Thread(target=app.run).start()
time.sleep(1)

In [ ]:
app.loop()
te = pd.DataFrame(app.list_)
k = list(set(te['symbol']))
len(te), len(k)

In [ ]:
te_f = te.drop_duplicates(subset=["symbol", "date"])
te_f.to_parquet("2025_v1_tickers_test.parquet")
len(te), len(te_f)

In [ ]:
# test for missed symbols
k = list(set(te['symbol']))
for i in app.etfList[:]:
    if i not in k: print(i)

In [ ]:
app.loop()

In [ ]:
mycontract = Contract()
mycontract.symbol = "AAA"
mycontract.secType = "STK"
mycontract.exchange = "SMART"
mycontract.currency = "USD"
app.reqHistoricalData(app.nextId(), mycontract, "20250120 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
